# **fGraph 0 detection from eigenvalues**


<div id="import_libraries">
    <h2>Import Libraries</h2>
</div>


In [ ]:
#!pip install numpy
#!pip install pandas
#!pip install scikit-learn
#!pip install matplotlib
#!pip install xgboost

To import the libraries that will be used in this lab, execute the cells below. 


In [1]:
# Import the libraries we need to use in this lab
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import time
# model trainin set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
#models 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
#model evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score

## Load the dataset




In [2]:
# load the dataset
path= "den_Kirchhoff_10.csv"
# read the input data
df = pd.read_csv(path)
df


,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,DEN_EIGEN10,DEN_EIGEN11,DEN_EIGEN12,DEN_EIGEN13,DEN_EIGEN14
0,1,-9.328318e-16,1.980639,2.605225,2.935763,4.600013,5.235617,5.464770,6.196335,6.244244,7.000000,7.000000,7.000000,7.774571,7.962823
1,1,-1.366537e-16,1.980639,2.267949,2.935763,4.600013,5.000000,5.464770,5.732051,6.244244,7.000000,7.000000,7.000000,7.000000,7.774571
2,1,-1.457943e-15,1.893658,2.595900,2.667028,4.510499,4.700596,5.285004,6.156444,6.218458,6.675399,7.000000,7.000000,7.475932,7.821081
3,1,3.238516e-16,1.875854,2.222753,2.666361,4.452883,4.665045,5.000000,5.630163,6.208118,6.565654,7.000000,7.000000,7.000000,7.713168
4,1,2.175634e-15,1.792597,2.000000,2.657077,4.445480,4.529317,5.000000,5.000000,6.000000,6.475353,6.813607,7.000000,7.000000,7.286571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153247,0,1.129899e-15,0.964106,3.518806,3.625205,3.738198,4.769795,5.311108,5.660123,5.673447,6.650581,7.170086,7.253124,8.601679,9.063742
153248,0,-1.757850e-15,0.821939,3.518806,3.615524,3.738198,4.734859,5.311108,5.386536,5.660123,6.482115,6.803241,7.170086,8.155785,8.601679
153249,0,1.719407e-15,0.962744,3.310347,3.617231,3.726629,4.603344,5.039109,5.590112,5.669429,6.373674,6.736111,7.249703,8.426072,8.695495
153250,0,-5.992738e-16,0.960885,3.196959,3.538432,3.692236,4.467965,4.812580,5.476823,5.598530,6.191690,6.550124,7.249369,7.831946,8.432461


In [3]:
# Let's round up the columns to 8 decimals.
# Take all non-integer coulumns
cols_to_round = df.columns.difference(['COEFFICIENT_DEN'])

# Round those columns to, say, 8 decimals
df[cols_to_round] = df[cols_to_round].round(8)
df

,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,DEN_EIGEN10,DEN_EIGEN11,DEN_EIGEN12,DEN_EIGEN13,DEN_EIGEN14
0,1,-0.0,1.980639,2.605225,2.935763,4.600013,5.235617,5.464770,6.196335,6.244244,7.000000,7.000000,7.000000,7.774571,7.962823
1,1,-0.0,1.980639,2.267949,2.935763,4.600013,5.000000,5.464770,5.732051,6.244244,7.000000,7.000000,7.000000,7.000000,7.774571
2,1,-0.0,1.893658,2.595900,2.667028,4.510499,4.700596,5.285004,6.156444,6.218458,6.675399,7.000000,7.000000,7.475932,7.821081
3,1,0.0,1.875854,2.222753,2.666361,4.452883,4.665045,5.000000,5.630163,6.208118,6.565654,7.000000,7.000000,7.000000,7.713168
4,1,0.0,1.792597,2.000000,2.657077,4.445480,4.529317,5.000000,5.000000,6.000000,6.475353,6.813606,7.000000,7.000000,7.286571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153247,0,0.0,0.964106,3.518806,3.625205,3.738198,4.769795,5.311108,5.660123,5.673447,6.650581,7.170086,7.253124,8.601679,9.063742
153248,0,-0.0,0.821939,3.518806,3.615524,3.738198,4.734859,5.311108,5.386536,5.660123,6.482115,6.803241,7.170086,8.155785,8.601679
153249,0,0.0,0.962744,3.310347,3.617231,3.726629,4.603344,5.039109,5.590112,5.669429,6.373674,6.736111,7.249703,8.426072,8.695495
153250,0,-0.0,0.960885,3.196959,3.538432,3.692236,4.467965,4.812580,5.476823,5.598530,6.191690,6.550124,7.249369,7.831946,8.432461


# ChatGpt classifier computation

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# 1. Extract features and target
y = df["COEFFICIENT_DEN"]
X = df.drop(columns=["COEFFICIENT_DEN"])


In [5]:
# 2. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [6]:
# 3. Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train a classifier (Logistic Regression here)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_scaled, y_train)

# 5. Predict and evaluate
y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7849009820234251
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.87      0.84     20398
           1       0.71      0.61      0.65     10253

    accuracy                           0.78     30651
   macro avg       0.76      0.74      0.75     30651
weighted avg       0.78      0.78      0.78     30651



### Decision tree

In [7]:
# 1. (Optional) scale features – not needed for decision trees but harmless
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Train a Decision Tree classifier
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train_scaled, y_train)

# 3. Predict and evaluate
y_pred = tree_clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7326025252030929
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.80      0.80     20398
           1       0.60      0.60      0.60     10253

    accuracy                           0.73     30651
   macro avg       0.70      0.70      0.70     30651
weighted avg       0.73      0.73      0.73     30651



### Random forest

with adj matix acc was 79.59% now it is 80.4%

In [8]:
# 1. Train the Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=12)
rf_clf.fit(X_train_scaled, y_train)

# 2. Predict and evaluate
y_pred_rf = rf_clf.predict(X_test_scaled)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.8039868193533654
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.89      0.86     20398
           1       0.74      0.64      0.68     10253

    accuracy                           0.80     30651
   macro avg       0.79      0.76      0.77     30651
weighted avg       0.80      0.80      0.80     30651



In [10]:
# 3. Feature importances from the random forest
importances = rf_clf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
      feature  importance
1  DEN_EIGEN2    0.222414
2  DEN_EIGEN3    0.086996
3  DEN_EIGEN4    0.081696
6  DEN_EIGEN7    0.073708


# Can the model fit the data?

Let's use the 100% of our data to train the model. Does a solution exist? Let's try the logistic regression

In [11]:
# Prepare features and labels
X = df.drop(columns=["COEFFICIENT_DEN"])
y = df["COEFFICIENT_DEN"]

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train on full data
clf = LogisticRegression(max_iter=1000)
clf.fit(X_scaled, y)

# Evaluate on same data
y_pred = clf.predict(X_scaled)
print("Accuracy on full dataset:", accuracy_score(y, y_pred))


Accuracy on full dataset: 0.7825738000156605


In [ ]:
from sklearn.svm import SVC

# Train SVM on full dataset
svm_clf = SVC(kernel='rbf')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))


Polynomial fit

In [ ]:
# Train SVM on full dataset
svm_clf = SVC(kernel='poly')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))

SVM accuracy on full dataset: 0.7793296089385475


## Random forest  perfect score

We see that the random forest reach a prefect score. If we remove the denominator eigenvalues it reaces an almost perfect score

In [12]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_scaled, y)

y_pred_rf = rf_clf.predict(X_scaled)
print("Random Forest accuracy:", accuracy_score(y, y_pred_rf))

Random Forest accuracy: 0.9987993631404484


In [14]:
importances = rf_clf.feature_importances_
feat_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feat_importance = feat_importance.sort_values(by='importance', ascending=False)

print("Top features:\n", feat_importance.head())


Top features:
        feature  importance
1   DEN_EIGEN2    0.243562
2   DEN_EIGEN3    0.085731
3   DEN_EIGEN4    0.074342
6   DEN_EIGEN7    0.067766
9  DEN_EIGEN10    0.066197


In [15]:
df.corr()["COEFFICIENT_DEN"]

COEFFICIENT_DEN    1.000000
DEN_EIGEN1              NaN
DEN_EIGEN2         0.529580
DEN_EIGEN3        -0.241873
DEN_EIGEN4        -0.249583
DEN_EIGEN5         0.098025
DEN_EIGEN6         0.164744
DEN_EIGEN7         0.260760
DEN_EIGEN8         0.226650
DEN_EIGEN9         0.254778
DEN_EIGEN10        0.268501
DEN_EIGEN11        0.230558
DEN_EIGEN12        0.151409
DEN_EIGEN13        0.054699
DEN_EIGEN14        0.022029
Name: COEFFICIENT_DEN, dtype: float64

In [16]:
# 5. Find misclassified samples
misclassified = df[y != y_pred_rf]

print(f"Number of misclassified samples: {len(misclassified)}")
print("Misclassified indices:")
print(misclassified.index.tolist())

Number of misclassified samples: 184
Misclassified indices:
[217, 789, 795, 799, 825, 4002, 4159, 4162, 4965, 4998, 5674, 6165, 6204, 6218, 6228, 6242, 11644, 20307, 20542, 20674, 21414, 23585, 23731, 24050, 24052, 24056, 24094, 24573, 24581, 25117, 26064, 26231, 26233, 26265, 26827, 26828, 26833, 26837, 28491, 28640, 29690, 30110, 31600, 32340, 32341, 33027, 33172, 33177, 33557, 34415, 34442, 34995, 35026, 36815, 38230, 38442, 38649, 38650, 38651, 38984, 39042, 39156, 41696, 42874, 42890, 43065, 43504, 43511, 43831, 44628, 46600, 46695, 46698, 46982, 47509, 47620, 47681, 47828, 47846, 48201, 49935, 49938, 50248, 50265, 51703, 52138, 52829, 52832, 52847, 52912, 53175, 53305, 53310, 53467, 53660, 54394, 55233, 57153, 58133, 58830, 60154, 61014, 62946, 62954, 62974, 63001, 63006, 63142, 63145, 63179, 63219, 63227, 63436, 67072, 67077, 68503, 68532, 69728, 70003, 70018, 70020, 70037, 71209, 78166, 78168, 78170, 78886, 79777, 79782, 79849, 80466, 80487, 81096, 81994, 83398, 84792, 84793, 8